In [1]:
import requests
import re
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
# Define the url we are going to take the info from

url4 = 'https://en.wikipedia.org/wiki/Critics%27_Choice_Movie_Award_for_Best_Supporting_Actress'

In [3]:
# Make the request to the url

res = requests.get(url4)

# Check if all went well

res.status_code

200

In [4]:
# 'html.parser' is used as the parser for the BeautifulSoup library.

soup = BeautifulSoup(res.content, 'html.parser') 

# Extract the name of the tables:

tables = soup.find_all("table", {"class": "wikitable"})

# Get the number of tables in the web, in this case 4 in this case

len(tables)

4

In [5]:
# Get the info of every table

soup1 = tables[0].find_all("td")
soup2 = tables[1].find_all("td")
soup3 = tables[2].find_all("td")
soup4 = tables[3].find_all("td")

# Put all the info together

the_soup = soup1 + soup2 + soup3 + soup4

In [6]:
# Transform the data extacted into text
# Iterating over the outer list

actors1 = [x.text for item in the_soup  

                # Iterating over the inner lists
                
               for x in item]

In [7]:
# The re.sub pattern '\[(.*?)\]' is used to match the elements between the square brackets and and replace it with a ""

actors1 = [re.sub(r'\[(.*?)\]', '', item) for item in actors1]

# This re. sub pattern '\((.*?)\)' is used to match the elements between the parenthesis and replace it with a ""

actors1 = [re.sub(r'\((.*?)\)', '', item) for item in actors1]

# This takes out the empty spaces from our dataset

actors1 = [item for item in actors1 if item.strip("")]

In [8]:
# Iterate over the elements of the actors1 list

for i, item in enumerate(actors1):

    # Check if the current element is the selected one

    if actors1[i] == 'Elaine Miller / Sara Gaskell':

        # Replace the current element

        actors1[i] = 'Elaine Miller'

        # Use slicing to insert the new elements at the current index

        actors1[i+2:i] = ["\n", "Frances McDormand", "Sara Gaskell", "Wonder Boys"]

In [9]:
# Create a dictionary of replacements

replacements = {'Almost Famous / Wonder Boys': 'Almost Famous',
                'Chieko Wataya\n': 'Chieko Wataya','Natalie Keener\n':'Natalie Keener',
                'Joy Wang / Jobu Tupaki\n':'Joy Wang / Jobu Tupaki'}

# Iterate over the list and replace values with values from the dictionary

for i in range(len(actors1)):
    if actors1[i] in replacements:
        actors1[i] = replacements[actors1[i]]

In [10]:
# Iterate over the elements of the actors1 list

for i, item in enumerate(actors1):

    # Check if the current index is not the last element of the list, and add the missing information

    if i < len(actors1)-1:
        if item == '\n':
            if actors1[i+1] == 'Rinko Kikuchi' and actors1[i+2] == 'Chieko Wataya':
                actors1.insert(i+3, 'Babel')
                actors1.insert(i+4, "\n")
            elif actors1[i+1] == 'Anna Kendrick' and actors1[i+2] == 'Natalie Keener':
                actors1.insert(i+3, "Up in the Air")
                actors1.insert(i+4, "\n")
            elif actors1[i+1] == 'Rachel Weisz' and actors1[i+2] == 'Sarah Churchill':
                actors1.insert(i+3, 'The Favourite')
            elif actors1[i+1] == 'Stephanie Hsu' and actors1[i+2] == 'Joy Wang / Jobu Tupaki':
                actors1.insert(i+3, 'Everything Everywhere All at Once')
                actors1.insert(i+4, "\n")

In [11]:
# Create a new dataframe with columns "year_ceremony", "film", and "name"

df2 = pd.DataFrame(columns=["year_ceremony", "film", "name", "character"])

# Initialize empty variables to store the year, film, and name

year = ""
film = ""
name = ""
character = ""

# Iterate through the new_data list, using the enumerate function to keep track of the index

for i, element in enumerate(actors1):

    # Check if the element contains a newline character

    if "\n" in element: 

        # If it does, append the current year, film, and name to the dataframe

        df2 = df2.append({"year_ceremony": year, "film": film, "name": name, "character": character}, ignore_index=True)

        # Update the film, name and year variables with the new values

        film = actors1[i-1]
        name = actors1[i-3]
        year = actors1[i-4]
        character = actors1[i-2]
    if i == len(actors1) - 1:
    
        # if it's the last element append the last year, film, and name to the dataframe
    
        df2 = df2.append({"year_ceremony": year, "film": film, "name": name, "character": character}, ignore_index=True)

C:\Users\juano\AppData\Local\Temp\ipykernel_11268\3299305245.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name, "character": character}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_11268\3299305245.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name, "character": character}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_11268\3299305245.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df2 = df2.append({"year_ceremony": year, "film": film, "name": name, "character": character}, ignore_index=True)
C:\Users\juano\AppData\Local\Temp\ipykernel_11268\32

In [12]:
# Iterate through the elements in 'df2'

for years in range(1, len(df2)):

    # Check if the current element is a newline character

    if df2.iloc[years]['year_ceremony'] == '\n':

        # Replace the current element with the value of the previous element

        df2.at[years, 'year_ceremony'] = df2.iloc[years-1]['year_ceremony']

In [13]:
# Drop the first row (index 0) from the DataFrame because is empty

df2 = df2.drop(df2.index[0])

In [14]:
# Use the assign method to add a new column called "category" and the award name "cca"

df2 = df2.assign(category='Actor in a Supporting Role').assign(cca='Nominee')

# Create a list of columns in the desired order

columns = ['year_ceremony', 'category', 'film', 'name', 'cca']

# Reorder columns

df2 = df2[columns]

In [15]:
# Extract the info of the winners:

cca_winners = soup.find_all("td", {"style":"background:#B0C4DE;"})

# Get the number of the award winners

len(cca_winners)

90

In [16]:
# Extract the text from cca_winners

cca_winners = [x.text for x in cca_winners]

# Split by "\n" and get the element first element

cca_winners= [x.split("\n")[0] for x in cca_winners]

In [17]:
# Iterate for every element in the cca_winners

for match in cca_winners:

    # If a match is found between the list and the columns, change from Nominnee to Winner

    df2.loc[df2["film"] == match, 'cca'] = 'Winner' 

In [18]:
# Use the & operator to combine the conditions

condition = (df2['film'].isin(["Almost Famous", "Wonder Boys"])) & (df2['name'] == "Frances McDormand")

# Select the rows that match the condition and update the 'cca' column to "Winner"

df2.loc[condition, 'cca'] = "Winner"

In [19]:
# Convert to csv and store it in data folder

df2.to_csv("../data/Actress_In_A_Supporting_Role_soup_cca.csv")